# Lending Clug Case Study: Credit Risk
description...

## Imports & Settings

In [1]:
import pandas as pd

In [2]:
# for better visibility, we will display the whole dataframe and show floats with 2 decimal places
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## First Look

In [3]:
df = pd.read_csv('loan.csv')
df.set_index('id', inplace=True)
df.head(5)

/Users/houghton/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1077501,1296599,5000,5000,4975.00,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.00,Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0,Jan-85,1,NaN,NaN,3,0,13648,83.70%,9,f,0.00,0.00,5863.16,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-15,171.62,NaN,May-16,0.00,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN
1077430,1314167,2500,2500,2500.00,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.00,Source Verified,Dec-11,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0,Apr-99,5,NaN,NaN,3,0,1687,9.40%,4,f,0.00,0.00,1008.71,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,NaN,Sep-13,0.00,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN
1077175,1313524,2400,2400,2400.00,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.00,Not Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,real estate business,606xx,IL,8.72,0,Nov-01,2,NaN,NaN,2,0,2956,98.50%,10,f,0.00,0.00,3005.67,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-14,649.91,NaN,May-16,0.00,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN
1076863,1277178,10000,10000,10000.00,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.00,Source Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0,Feb-96,1,35.00,NaN,10,0,5598,21%,37,f,0.00,0.00,12231.89,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-15,357.48,NaN,Apr-16,0.00,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,N

## Data Clean Up

While importing, Pandas already warns us that there are columns with mixed data types. 
> 'DtypeWarning: Columns (47) have mixed types.'

For the analysis we need our data to be correctly formatted (preferably numeric).

### Empty & Monotonous Columns
If the data contains completely empty columns we can drop them as they don't contain any information to analyze.

In [4]:
empty_cols = [col for col in df.columns if df[col].isnull().all()]
print(len(empty_cols), 'empty columns found')
print('Empty columns:', empty_cols)

df.drop(empty_cols, axis=1, inplace=True)

54 empty columns found
Empty columns: ['mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'tot_hi_cred_lim', 'total_bal_ex_mort', 't

Columns that contain nothing but one value don't add any information and can be droped. We also look at columns with 2 values to see if they can be dropped.

In [5]:
cols_with_1_value = [col for col in df.columns if df[col].nunique() == 1]
df.drop(cols_with_1_value, axis=1, inplace=True)


cols_with_2_values = [col for col in df.columns if df[col].nunique() == 2]
for col in cols_with_2_values:
    print(col, df[col].unique())

term [' 36 months' ' 60 months']
next_pymnt_d [nan 'Jun-16' 'Jul-16']


In [6]:
df.next_pymnt_d.value_counts()

Jun-16    1125
Jul-16      15
Name: next_pymnt_d, dtype: int64

Just the 'term' (duration of the loan) (36 or 60 months) and the 'next_pymnt_d' (June or July 2016 if any) have only 2 different values. 
<br>Duration might be a factor associated with credit risk, so we leave that column in.
<br>We assume that the cut-off date was in or before June 2016, so that the entries in 'next_pymnt_d' were in the future when the data was taken. This would explain why there are so few and it fits with the time frame of the project description 
(2011 + 60 months = 2017). We don't know if these payments happened but might need the information and therefor leave the column in.

### Units

We can convert units that are spelled out in each cell to numeric values

In [7]:
# converting all columns with units in in values to numbers
df['term'] = df['term'].str.replace(' months', '').astype(int)
df.rename(columns={'term': 'term_in_months'}, inplace=True)

df['int_rate'] = df['int_rate'].str.replace('%', '').astype(float)
df['revol_util'] = df['revol_util'].str.replace('%', '').astype(float)

In [8]:
df.emp_length.unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', nan],
      dtype=object)

We see above that the employment length has 3 vaules (<1,1 and 10+) that don't end in 'years'. We can convert these.


In [9]:
df['emp_length'] = df['emp_length'].replace('10+ years', 10).astype(str)
df['emp_length'] = df['emp_length'].replace('< 1 year', 0.5).astype(str)
df['emp_length'] = df['emp_length'].replace('1 year',     1).astype(str)

df['emp_length'] = df['emp_length'].str.replace(' years', '').astype(float)


### Dates

In [10]:
# Looking at the table we can see the following columns contain dates
cols_with_date = [
                  'issue_d', 'earliest_cr_line',
                  'last_pymnt_d', 'next_pymnt_d', 
                  'last_credit_pull_d'
                  ]

for col in cols_with_date:
    df[col] = pd.to_datetime(df[col], format='%b-%y')


@Justin: do you know a way of detecting dates other than 'Looking at the table' 😅

### Description Column

The description column ('desc') contains unstructured data. Let's see if we can extract usefull information from it.

In [11]:
print(len(df.desc.unique()), 'unique descriptions')

print('Examples:')
for desc in df.sample(5).desc:
    print('-', str(desc).strip())


26528 unique descriptions
Examples:
- nan
- nan
- Borrower added on 03/08/11 > This will be my first personal sailboat. I already practice and race on a J24 which is owned by a coworker, but it is hard to work around her schedule at times. This boat will allow me to practice on my own time and improve my skills for regattas.<br/>
- Borrower added on 05/25/11 > Consolidate Family Bills of Citibank Student Loan, HSBC Retail Services, GE Money Bank Card, Chase Mastercard<br/>
- nan


The descriptions often contain the date when the borrower was added and their usecase. Although the usecase might be interesting for further analysis we only extract the date so we can use it in the bivariate analysis later.

In [12]:
# extract the date from the description if the description starts with 'Borrower added on'
def extract_added_date(desc):
    desc = str(desc).strip()
    if desc.startswith('Borrower added on'):
        return desc.split(' >')[0][-8:]
    else:
        return None

# insert the new column next to the description column and convert it to datetime
df.insert(df.columns.get_loc('desc') + 1,'borrower_added_d',df.desc.apply(extract_added_date))
df['borrower_added_d'] = pd.to_datetime(df['borrower_added_d'], format='%m/%d/%y', errors='coerce')

In [13]:
# The web addresses of the URL column are inaccessible and therefore beyond the scope of this assignment.
df.drop(['url'], axis=1, inplace=True)

### Grades

@Justin: do you have an more elegant solution for this?

In [14]:
# Converting the sub_grade scale to categorical numberical values

df.sub_grade.astype("category").cat.codes

id
1077501     6
1077430    13
1077175    14
1076863    10
1075358     9
           ..
92187       3
90665      10
90395       3
90376       1
87023      21
Length: 39717, dtype: int8

In [15]:
# # Converting the sub_grade scale to numeric from A1 = 1 to G5 = 35

# grade_scale = {
#     'A1': 1, 'A2': 2, 'A3': 3, 'A4': 4, 'A5': 5,
#     'B1': 6, 'B2': 7, 'B3': 8, 'B4': 9, 'B5': 10,
#     'C1': 11,'C2': 12,'C3': 13,'C4': 14,'C5': 15,
#     'D1': 16,'D2': 17,'D3': 18,'D4': 19,'D5': 20,
#     'E1': 21,'E2': 22,'E3': 23,'E4': 24,'E5': 25,
#     'F1': 26,'F2': 27,'F3': 28,'F4': 29,'F5': 30,
#     'G1': 31,'G2': 32,'G3': 33,'G4': 34,'G5': 35
# }

# df.insert(df.columns.get_loc('sub_grade') + 1,'grade_num',df['sub_grade'].map(grade_scale))

## Overview

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39717 entries, 1077501 to 87023
Data columns (total 47 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   member_id                39717 non-null  int64         
 1   loan_amnt                39717 non-null  int64         
 2   funded_amnt              39717 non-null  int64         
 3   funded_amnt_inv          39717 non-null  float64       
 4   term_in_months           39717 non-null  int64         
 5   int_rate                 39717 non-null  float64       
 6   installment              39717 non-null  float64       
 7   grade                    39717 non-null  object        
 8   sub_grade                39717 non-null  object        
 9   emp_title                37258 non-null  object        
 10  emp_length               38642 non-null  float64       
 11  home_ownership           39717 non-null  object        
 12  annual_inc               3

In [17]:
df.describe()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term_in_months,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,pub_rec_bankruptcies
count,39717.00,39717.00,39717.00,39717.00,39717.00,39717.00,39717.00,38642.00,39717.00,39717.00,39717.00,39717.00,14035.00,2786.00,39717.00,39717.00,39717.00,39667.00,39717.00,39717.00,39717.00,39717.00,39717.00,39717.00,39717.00,39717.00,39717.00,39717.00,39717.00,39020.00
mean,850463.56,11219.44,10947.71,10397.45,42.42,12.02,324.56,5.03,68968.93,13.32,0.15,0.87,35.90,69.70,9.29,0.06,13382.53,48.83,22.09,51.23,50.99,12153.60,11567.15,9793.35,2263.66,1.36,95.22,12.41,2678.83,0.04
std,265678.31,7456.67,7187.24,7128.45,10.62,3.72,208.87,3.48,63793.77,6.68,0.49,1.07,22.02,43.82,4.40,0.24,15885.02,28.33,11.40,375.17,373.82,9042.04,8942.67,7065.52,2608.11,7.29,688.74,148.67,4447.14,0.20
min,70699.00,500.00,500.00,0.00,36.00,5.42,15.69,0.50,4000.00,0.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00,0.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,666780.00,5500.00,5400.00,5000.00,36.00,9.25,167.02,2.00,40404.00,8.17,0.00,0.00,18.00,22.00,6.00,0.00,3703.00,25.40,13.00,0.00,0.00,5576.93,5112.31,4600.00,662.18,0.00,0.00,0.00,218.68,0.00
50%,850812.00,10000.00,9600.00,8975.00,36.00,11.86,280.22,4.00,59000.00,13.40,0.00,1.00,34.00,90.00,9.00,0.00,8850.00,49.30,20.00,0.00,0.00,9899.64,9287.15,8000.00,1348.91,0.00,0.00,0.00,546.14,0.00
75%,1047339.00,15000.00,15000.00,14400.00,60.00,14.59,430.78,9.00,82300.00,18.60,0.00,1.00,52.00,104.00,12.00,0.00,17058.00,72.40,29.00,0.00,0.00,16534.43,15798.81,13653.26,2833.40,0.00,0.00,0.00,3293.16,0.00
max,1314167.00,35000.00,35000.00,35000.00,60.00,24.59,1305.19,10.00,6000000.00,29.99,11.00,8.00,120.00,129.00,44.00,4.00,149588.00,99.90,90.00,6311.47,6307.37,58563.68,58563.68,35000.02,23563.68,180.20,29623.35,7002.19,36115.20,2.00
